In [3]:
import numpy as np
from onix.units import ureg, Q_
from typing import Optional, Union

In [4]:
class Photodiode:
    """Photodiode shot noise calculation.

    Args:
        R: photocurrent responsivity at the target wavelength, A/W. This number does not account for amplification in an APD.
        M: APD photoelectron amplification factor. For other photodiodes, it should be 1.
        G: transimpedance gain from current to output voltage.
        maximum: Used to calculate the maximum input optical power depending on the input units:
                            - units of [electric potential], maximum is the maximum output voltage;
                            - units of [current], maximum is the maximum output current;
                            - units of [power], maximum is the maximum input optical power.
        NEP: based on Thorlabs definition (https://www.thorlabs.com/images/TabImages/Noise_Equivalent_Power_White_Paper.pdf)
             the input signal power that results in an SNR of 1 at 1 Hz output bandwidth. Thorlabs uses "optical NEP" but assumes
             it is equal to the "electrical NEP". Used to determine the electrical noise.
    """
    def __init__(self, R: Q_, R_max: Q_, NEP_min: Q_, M: float = 1, G: Q_ = 50 * ureg.V / ureg.A, maximum: Optional[Q_] = None):
    # def __init__(self, R: Q_, M: float, G: Q_, max_V_out: Optional[Q_] = None):
    # def __init__(self, R: Q_, M: float, G: Q_, max_V_out: Q_, NEP: Q_):
        self._P = None
        self._R = R
        self._R_max = R_max
        self._NEP_min = NEP_min
        self._M = M
        self._G = G

        self.max_P = self._max_P(maximum)

    def _max_P(self, maximum) -> Union[None, Q_]:
        """Maximum input optical power based on a maximum input/output parameter."""
        if maximum.check("[electric_potential]"):
            return (maximum / (self._G * self._M * self._R)).to("W")
        elif maximum.check("[power]"):
            return maximum.to("W")
        else:
            return None
    
    @property
    def P(self):
        """Input power."""
        return self._P

    @P.setter
    def P(self, P: Q_):
        """Input power."""
        if self.max_P is not None and P > self.max_P:
            print("Set power is higher than the maximum input power.")
        self._P = P

    @property
    def I_p(self) -> Q_:
        """Photocurrent before amplification."""
        return (self._P * self._R).to("A")

    @property
    def I_amp(self) -> Q_:
        """Photodiode current after amplification."""
        return self.I_p * self._M

    @property
    def V_out(self) -> Q_:
        """Photodiode output voltage."""
        return (self.I_amp * self._G).to("V")

    @property
    def shot_noise_I_p(self) -> Q_:
        """Noise spectral density of the photocurrent."""
        return (np.sqrt(2 * (1.6e-19 * ureg.A * ureg.s) * self.I_p)).to("A/Hz^(1/2)")

    @property
    def shot_noise_I_amp(self) -> Q_:
        """Noise spectral density of the amplified photodiode current."""
        return (self._M * self.shot_noise_I_p).to("A/Hz^(1/2)")

    @property
    def shot_noise_V_out(self) -> Q_:
        """Noise spectral density of the photodiode output voltage."""
        return (self._G * self.shot_noise_I_amp).to("V/Hz^(1/2)")

    def shot_noise_SNR(self, BW: Q_) -> Q_:
        """SNR for a certain bandwidth."""
        return (self._G * self.I_amp / self.shot_noise_V_out / np.sqrt(BW)).to("").magnitude

    @property
    def electrical_noise_V_out(self) -> Q_:
        """Electrical noise in V from NEP based on Thorlabs definition; assuming BW = 1 Hz."""
        return (self._NEP_min * self._R_max * self._M * self._G).to("V/Hz^(1/2)")

    def electrical_noise_SNR(self, BW: Q_) -> Q_:
        """ """
        return (self.V_out / self.electrical_noise_V_out / np.sqrt(BW)).to("").magnitude

    @property
    def total_noise_V_out(self) -> Q_:
        return np.sqrt(self.shot_noise_V_out ** 2 + self.electrical_noise_V_out ** 2)
        
    def total_noise_SNR(self, BW) -> Q_:
        return (self.V_out / self.total_noise_V_out / np.sqrt(BW)).to("").magnitude
        
        

In [5]:
# # APD130A

# M = 50
# R = 12 * ureg.A / ureg.W / M
# G = 50 * ureg.kV / ureg.A
# maximum_current = 100 * ureg.mA
# APD130A = Photodiode(R, M, G, maximum_current)

# # NEP Calc
# NEP_max = 0.2 * ureg.pW / np.sqrt(1 * ureg.Hz)
# R_max = 25 * ureg.A / ureg.W / 50
# BW = 50 * ureg.MHz
# # V = APD130A.electrical_noise(NEP_max, R_max, BW)
# P = APD130A.electrical_noise_P(NEP_max, R_max, BW)

# print(P)

# APD130A.P = 1.2 * ureg.uW
# APD130A.SNR(BW)
# APD130A.V_out
# APD130A.V_out/APD130A.SNR(BW)
# APD130A.N_V_out

In [9]:
BW = 1 * ureg.Hz
upper_limit_power = 5000 * ureg.uW

photodiodes = {
    "APD130A": {
        "M" : 50,
        "R" : 12 * ureg.A / ureg.W / 50,
        "R_max" : 25 * ureg.A / ureg.W / 50,
        "G" : 50 * ureg.kV / ureg.A,
        "limit" : 1.8 * ureg.V,
        "NEP_min" : 0.2 * ureg.pW / np.sqrt(1 * ureg.Hz),
    },
    # "PDA36A2-0dB": {
    #     "M" : 1,
    #     "R" : 0.25 * ureg.A / ureg.W,
    #     "R_max" : 0.65 * ureg.A / ureg.W,
    #     "G" : 0.75 * ureg.kV / ureg.A,
    #     "limit" : 5 * ureg.V,
    #     "NEP_min" : 75.7 * ureg.pW / np.sqrt(1 * ureg.Hz),
    # },
    # "PDA36A2-50dB": {
    #     "M" : 1,
    #     "R" : 0.25 * ureg.A / ureg.W,
    #     "R_max" : 0.65 * ureg.A / ureg.W,
    #     "G" : 2.38e2 * ureg.kV / ureg.A,
    #     "limit" : 5 * ureg.V,
    #     "NEP_min" : 3.69 * ureg.pW / np.sqrt(1 * ureg.Hz),
    # },
    "PDA36A2-20dB": {
        "M" : 1,
        "R" : 0.25 * ureg.A / ureg.W,
        "R_max" : 0.65 * ureg.A / ureg.W,
        "G" : 7.5 * ureg.kV / ureg.A,
        "limit" : 5 * ureg.V,
        "NEP_min" : 3.4 * ureg.pW / np.sqrt(1 * ureg.Hz),
    },

    "PDA8A2": {
        "M" : 1,
        "R" : 0.31 * ureg.A / ureg.W,
        "R_max" : 0.56 * ureg.A / ureg.W,
        "G" : 50 * ureg.kV / ureg.A,
        "limit" : 1.4 * ureg.V,
        "NEP_min" : 7.8 * ureg.pW / np.sqrt(1 * ureg.Hz),
    },
    # OBSOLETE
    # "DET36A": {
    #     "M" : 1,
    #     "R" : 0.25 * ureg.A / ureg.W,
    #     "R_max" : 0.65 * ureg.A / ureg.W,
    #     "G" : 50 * ureg.V / ureg.A,
    #     "limit" : 100 * ureg.mA,
    #     "NEP_max" : 1.6e-14 * ureg.W / np.sqrt(1 * ureg.Hz),
    # }
}

In [10]:
for photodiode in photodiodes:

    # Noise calculations
    pd = Photodiode(photodiodes[photodiode]["R"],
                      photodiodes[photodiode]["R_max"],
                      photodiodes[photodiode]["NEP_min"],
                      photodiodes[photodiode]["M"],
                      photodiodes[photodiode]["G"],
                      photodiodes[photodiode]["limit"],
                     )

    # Check power
    if pd.max_P > upper_limit_power:
        pd.P = upper_limit_power
    else:
        pd.P = pd.max_P
    
    photodiodes[photodiode]["power"] = pd.P
    photodiodes[photodiode]["output voltage"] = pd.V_out

    # Shot noise
    photodiodes[photodiode]["shot noise voltage"] = pd.shot_noise_V_out
    photodiodes[photodiode]["shot noise SNR"] = pd.shot_noise_SNR(BW)

    # Electrical noise
    photodiodes[photodiode]["electrical noise voltage"] = pd.electrical_noise_V_out
    photodiodes[photodiode]["electrical noise SNR"] = pd.electrical_noise_SNR(BW)
    
    # Total noise
    photodiodes[photodiode]["total noise voltage"] = pd.total_noise_V_out
    photodiodes[photodiode]["total noise SNR"] = pd.total_noise_SNR(BW)

for i, photodiode in enumerate(photodiodes):
    print(f"\n------------------------------------------------------------------------------------------\n{list(photodiodes.keys())[i]}\n------------------------------------------------------------------------------------------")
    for k, v in photodiodes[photodiode].items():
        print(f"\n{k.rjust(30)} = {v}")
    print("\n\n\n")


------------------------------------------------------------------------------------------
APD130A
------------------------------------------------------------------------------------------

                             M = 50

                             R = 0.24 ampere / watt

                         R_max = 0.5 ampere / watt

                             G = 50.0 kilovolt / ampere

                         limit = 1.8 volt

                       NEP_min = 0.2 picowatt / hertz ** 0.5

                         power = 3e-06 watt

                output voltage = 1.8 volt

            shot noise voltage = 1.2e-06 volt / hertz ** 0.5

                shot noise SNR = 1500000.0

      electrical noise voltage = 2.5000000000000004e-07 volt / hertz ** 0.5

          electrical noise SNR = 7199999.999999999

           total noise voltage = 1.2257650672131262e-06 volt / hertz ** 0.5

               total noise SNR = 1468470.6296064076





-----------------------------------------------